# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv

from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****_WcA
LANGSMITH_API_KEY=****de27
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [3]:
from langchain.agents import create_agent

In [4]:
agent = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

I tried to start a hide-and-seek league—but good players are hard to find.


## values
You have seen this streaming mode in our examples so far. 

In [ ]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

I ordered a chicken and an egg from Amazon. I’ll let you know.


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [7]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

In our house, morning is a smoothie of sound,
Toast pops up cheering, “I’m bready for my round.”
The cat tiptoes softly like a whiskered hush,
The dog sends wag-mail with a tail-thump rush.

Someone tells a joke—whoever’s on duty—
We groan on purpose; that’s part of the beauty.
A grown-up flips pancakes like sunny-side moons,
Turning yawns into grins with spatula tunes.

Grandpa texts three emojis, bold and true:
Banana, rocket, waffle—his classic haiku.
The baby laughs hiccups, a squeaky kazoo,
Giggles go viral; we catch them too.

At dinner we pass carrots baton-style fast,
Broccoli wears mustaches, trying to pass.
We bargain with peas: “Two bites, then you’re free,”
They roll in agreement (politely) to me.

The couch keeps secrets, remote in its lair,
We file a bug report with cushion repair.
The Wi-Fi works best if you sit on the stairs,
But love is the hotspot that follows us everywhere.

When rain drums the roof, we camp in the den,
Tell stories that loop, then start again.
We co

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [42]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='0f34f5dd-8fa5-447d-9187-02fa20ba9b06')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='0f34f5dd-8fa5-447d-9187-02fa20ba9b06'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 132, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cl5SjlbcHp49AKCIBXx28nCNWJwWS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2f8ef95d-3e08-421c-a4cd-376652255a06-0', tool_calls=[{'name': 'get_we

In [9]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco, CA')
('custom', 'Acquired data for city: San Francisco, CA')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [21]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Q7nTyWIyJZeISrUkj43T7r0a)
 Call ID: call_Q7nTyWIyJZeISrUkj43T7r0a
  Args:
    city: San Francisco
================================= Tool Message =================================
Name: get_weather

It's always sunny in San Francisco!
================================== Ai Message ==================================

According to the weather tool: "It's always sunny in San Francisco!" 

Would you like current temperature, a multi-day forecast, or weather alerts for SF?


In [51]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in Guatemala City?"}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

It's always sunny in Guatemala City!It's always sunny in Guatemala City!

Would you like the current temperature, a multi-day forecast, or weather for a different location?

In [60]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in Guatemala City?"}]}
)

print(result["messages"][-1].content)

The weather service I checked returns: “It’s always sunny in Guatemala City!”

Would you like current temperature, humidity, wind, or a multi-day forecast? If so, tell me your preferred units (°C or °F) and I’ll fetch those details.
